In [ ]:
import json
from itertools import groupby

In [ ]:
pip install flatten_json

In [ ]:
from flatten_json import flatten

In [ ]:
json_file=json.load(open('sample-dataset-3 - Copy.json'))
json_file

In [ ]:
#1.Create a function that transforms raw transaction data
# Flatten nested JSON structures using lambda functions
flat_json = flatten(json_file)
print(flat_json)


In [ ]:
#Extract unique product categories using map() and set()
unique_categories = set(map(lambda x: x['Category'],'product_id'))

In [ ]:
#2. Create analysis functions
 
#Group transactions by region and calculate regional sales
grouped_dict = {}
for key, value in json_file.items():
    if value not in grouped_dict:
        grouped_dict[value] = [key]
    else:
        grouped_dict[value].append(key)

In [ ]:
#Find top-selling products using sorted() with custom key
res = {i: [j[0] for j in j] for i, j in groupby(sorted(json_file.items(), key = lambda x : x[1]), lambda x : x[1])}

In [ ]:
#Calculate average transaction value by payment method
average_value = json_file.groupby('Payment_Method')['Amount'].mean()

In [ ]:
#3.Create a report generation function 

#Filters completed transactions using filter()
completed_transactions = list(filter(lambda x: x['Status'] == 'Completed', json_file))

In [ ]:
#Sorts data by multiple criteria using lambda
sorted_data = sorted(json_file, key=lambda x: (x['category'],x['quantity']))

In [ ]:
#Generates summary statistics for different time periods
import statistics
mode_x = statistics.mode(i['x'] for i in json_file)
print(mode_x)